Import Library

In [1]:
import numpy as np
import cv2
import pandas as pd
import seaborn as sns
import os
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.optimizers import Adam, SGD, RMSprop
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
import scipy

Prepare Data Generator

In [2]:
train_dir = 'data/train'
val_dir = 'data/val'
test_dir= 'data/test/'

num_train = 28709
num_val = 3589
num_test = 3589
batch_size = 128
num_epoch = 20

train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(train_dir,
                                    target_size=(48, 48),
                                    batch_size=batch_size,
                                    shuffle=True,
                                    color_mode="grayscale",
                                    class_mode='categorical')

validation_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        val_dir,
        target_size=(48, 48),
        batch_size=batch_size,
        shuffle=True,
        color_mode="grayscale",
        class_mode='categorical')

test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_dir,
        target_size=(48, 48),
        batch_size=batch_size,
        shuffle=True,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


In [3]:
Model_Checkpoint_Callback = ModelCheckpoint('best_model.h5', 
                                            monitor='val_accuracy', 
                                            verbose=1, 
                                            save_best_only=True, 
                                            mode='max',
                                            save_freq = "epoch")
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                              factor=0.75,
                              patience=5,
                              verbose=1,
                              mode='max')

Model Building

In [4]:
model = Sequential()

model.add(Conv2D(64,(3,3),padding = 'same',input_shape = (48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(64,(3,3),padding = 'same',input_shape = (48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(128,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(128,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(256,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(256,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dropout(0.2))

model.add(Dense(7, activation='softmax'))
sgd = SGD(lr=0.01, weight_decay=0.0001, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=num_train//batch_size,
                              epochs=num_epoch,
                              validation_data=validation_generator,
                              validation_steps=num_val//batch_size,
                              callbacks=[Model_Checkpoint_Callback,reduce_lr]
                              )

C:\Users\Tommy\AppData\Local\Temp\ipykernel_18932\2010626827.py:41: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator=train_generator,


Epoch 1/20
224/224 [==============================] - ETA: 0s - loss: 3.4632 - accuracy: 0.2259
Epoch 1: val_accuracy improved from -inf to 0.24498, saving model to best_model.h5


C:\Users\Tommy\AppData\Roaming\Python\Python38\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


224/224 [==============================] - 695s 3s/step - loss: 3.4632 - accuracy: 0.2259 - val_loss: 1.8446 - val_accuracy: 0.2450 - lr: 0.0100
Epoch 2/20
  9/224 [>.............................] - ETA: 10:57 - loss: 2.5413 - accuracy: 0.2257

KeyboardInterrupt: 

In [5]:
from keras.models import load_model

model_test = load_model("best_model.h5")

In [6]:
from sklearn.metrics import classification_report, confusion_matrix
Y_pred = model_test.predict_generator(test_generator, num_test // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))

C:\Users\Tommy\AppData\Local\Temp\ipykernel_18932\4109819777.py:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model_test.predict_generator(test_generator, num_test // batch_size+1)


Confusion Matrix
[[  0   0   0 467   0   0   0]
 [  0   0   0  56   0   0   0]
 [  0   0   0 496   0   0   0]
 [  0   0   0 895   0   0   0]
 [  0   0   0 607   0   0   0]
 [  0   0   0 653   0   0   0]
 [  0   0   0 415   0   0   0]]


In [7]:
print('Classification Report')
target_names = list(train_generator.class_indices.keys())
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

Classification Report
              precision    recall  f1-score   support

       angry       0.00      0.00      0.00       491
     disgust       0.00      0.00      0.00        55
        fear       0.00      0.00      0.00       528
       happy       0.24      1.00      0.39       879
     neutral       0.00      0.00      0.00       626
         sad       0.00      0.00      0.00       594
    surprise       0.00      0.00      0.00       416

    accuracy                           0.24      3589
   macro avg       0.03      0.14      0.06      3589
weighted avg       0.06      0.24      0.10      3589



C:\Users\Tommy\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Tommy\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Tommy\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
